# Evaluacion del rendimiento del módelo

Vamos a utilizar la _**validacion de entrenamiento/prueba**_ <br><br>

La cual consiste en dividir el conjunto de datos en dos: _conjuntos de entrenamiento_ y _conjunto de prueba_.<br>
Siguiendo los siguintes pasos: <br>
- Generar el _Conjunto de entrenamiento_ (75% de el conjunto original).    
- Generar el _Conjunto de prueba_ (25% de el conjunto original).
- Utilizar el _conjunto de entrenamiento_ para predecir los valores del precio del _conjunto de prueba_.
- Comparar los valores predichos con los precios del _conjunto de prueba_.


In [34]:
import pandas as pd
import numpy as np

dc_listings = pd.read_csv('dc_airbnb.csv')

stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')

train_df = dc_listings.iloc[0:2792].copy()
test_df = dc_listings.iloc[2792:].copy()


In [35]:
def predict_price(new_listings):
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['accommodates'].apply( lambda x_acc: np.abs(x_acc - new_listings) )
    temp_df = temp_df.sort_values('distance')
    nearest_neighbor_prices = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbor_prices.mean()
    return (predicted_price)

In [36]:
test_df['predicted_price'] = test_df['accommodates'].apply( predict_price )
test_df.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state,predicted_price
2792,20%,75%,1,2,Entire home/apt,0.0,1.0,1.0,120.0,NaN,NaN,1,1125,8,38.922187,-77.032475,Washington,20009,DC,104.0
2793,100%,25%,2,3,Entire home/apt,2.0,2.0,1.0,140.0,$75.00,$150.00,2,1125,7,38.931681,-77.044739,Washington,20010,DC,201.2
2794,NaN,NaN,1,4,Entire home/apt,2.0,1.0,1.0,299.0,NaN,NaN,2,1125,5,38.933765,-77.031488,Washington,20010,DC,145.8
2795,100%,100%,1,3,Entire home/apt,1.0,1.0,1.0,85.0,$30.00,$250.00,1,92,2,38.925692,-77.032616,Washington,20009,DC,201.2
2796,100%,100%,1,6,Entire home/apt,2.0,2.0,3.0,175.0,$65.00,$850.00,1,1125,62,38.927572,-77.033604,Washington,20009,DC,187.2


## Metricas de error

Ahora necesitamos cuantificar la calidad de las predicciones. <br>
Esta _metrica de error_ cuantificará la inexacitud del modelo. <br> <br>

Para este caso indicara la diferencia entre los valores predichos y los valores reales del precio.

#### MAE (Mean Absolute Error)

Una metrica de error que nos ayudaria a ver un poco que tan bueno es nuestro modelo seria la del _error medio absoluto_.<br>
La cual se define como: $$ MAE = \displaystyle\frac{1}{n}\sum\limits_{i=1}^n|(actual_1 - predict_1)| + ... + |(actual_n - predict_n)|$$

In [37]:
test_df['error'] = np.abs( test_df['price'] - test_df['predicted_price'] )
mae = test_df['error'].mean()
print(f"El valor del MAE es: {mae}")

El valor del MAE es: 58.60794844253491


#### MSE (Mean Square Error)

El error medio al cuadrado (MSE) aclara la diferencia entre los valores predichos y los reales. <br>
Por ejemplo: una prediccion que se desvia en 100 dolares que tendra un error de 10,000 es 100 veces mayor que una prediccion <br>
que se desvia solo 10 dolares que tendra un error de 100.

El MSE esta definido por la siguiente ecuacion: $$ MSE = \frac{1}{n}\sum\limits_{k=1}^n(actual_1 - predicted_1)^2 + ... + (actual_n - predicted_n)^2 $$

Dode $n$ representa el numero de filas del conjunto de prueba.

In [38]:
test_df['squared_error'] = ( test_df['price'] - test_df['predicted_price'] )**(2)
mse = test_df['squared_error'].mean()
print(f"El valor del MSE es: {mse}")

El valor del MSE es: 19031.671493018257


### ¿Que nos dice esto sobre la calidada de las predicciones y el modelo?

Por sí mismo, el valor del error cuadratico medio de un solo modelo no es muy útil. Sin embargo, podemos entrenar otro <br>
modelo y luego comparar los valores del _MSE_ para ver cual modelo funciona mejor.

Entrenando ahora el modelo utilizando la columan _bathrooms_.

In [39]:
def predicted_price( new_listings ):
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['bathrooms'].apply( lambda x_baths: np.abs(x_baths - new_listings) )
    temp_df = temp_df.sort_values('distance')
    nearest_neighbors_prices =  temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbors_prices.mean()
    return (predicted_price)

In [42]:
test_df['predicted_price'] = test_df['bathrooms'].apply( predicted_price )
test_df['squared_error'] = (test_df['price'] - test_df['predicted_price'])**(2)
mse = test_df['squared_error'].mean()
print(f'El MSE usando la caracteristica "bathroom" es: {mse}')

El MSE usando la caracteristica "bathroom" es: 18405.44408163265


Aparentemente el segundo modelo es de mejor calidad. Ya que tiene un MSE mas chico.

## Raiz del Error Cuadratico Medio (RMSE)

Loa valores del _MSE_ aunque nos da una idea relativa de cual es el mejor modelo. No nos ayuda a entenrir si el rendiemnto es lo suficientemente <br>
bueno en general. Esto debido a que las unidades estan al cuadrado. En cambio, **la raiz del error cuadratico medio** cuyas unidades son la undad base <br>
(en este caso, dolares). Esta metrica esta definida de la siguiente manera: $$ RMSE = \sqrt{MSE} $$

Calculando el $ RMSE $ para la columna _bathrooms_ (segundo modelo):

In [43]:
rmse_bathrooms = mse ** (1/2)
print(f'El RMSE para el modelo que utiliza la caracteristica "bathrooms" es: {rmse_bathrooms}')

El RMSE para el modelo que utiliza la caracteristica "bathrooms" es: 135.66666532952246


## Comparacion del **MAE** Y EL **RMSE**

El _RMSE_ dio un valor de aproximadamente  135.6, por lo que debemos esperar que el modelo se equivoque en 135.6 dolares de media para los valores <br>
predichos. Este es un error medio muy alto dado el escenario a resolver. Asi que debemos reducir ese error. <br><br>

Para entender mejor un modelo especifico, podemos ocmparar multiples metricas de error para el mismo modelo. <br>
Si se observa la ecuacion del **MAE**: $$ MAE = \displaystyle\frac{1}{n}\sum\limits_{i=1}^n|(actual_1 - predict_1)| + ... + |(actual_n - predict_n)|$$ 
Observará que las diferencias entre los valores predichos y los reales crecen linealmente. Una prediccion que se desvia 10 dolares tiene un error 10 veces mayor que la prediccion que se desvia 1 dolar.<br><br>

Sin embargo, si observamos la ecuacion del RMSE: $$ RMSE = \sqrt{MSE} = \sqrt{\frac{\sum\limits_{k=1}^n(actual_1 - predicted_1)^2 + ... + (actual_n - predicted_n)^2}{n}} $$ <br>

Observará que los errores individuales crecen cuadraticamente y tienen un efecto diferente en el valor final del _RMSE_.

In [48]:
errors_one = pd.Series( [ 5 if num%2 == 0 else 10 for num in range(0,18) ] )
print(errors_one)

errors_two =  pd.Series( [ 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 1000 ] )

mae_one = errors_one.sum()/len(errors_one)
rmse_one = np.sqrt( (errors_one**2).sum()/len(errors_one) )
print(f"mae_one = {mae_one}")
print(f"rmse_one = {rmse_one}")

mae_two = errors_two.sum()/len(errors_two)
rmse_two = np.sqrt( (errors_two**2).sum()/len(errors_two) )
print(f"mae_two = {mae_two}")
print(f"rmse_two = {rmse_two}")

0      5
1     10
2      5
3     10
4      5
5     10
6      5
7     10
8      5
9     10
10     5
11    10
12     5
13    10
14     5
15    10
16     5
17    10
dtype: int64
mae_one = 7.5
rmse_one = 7.905694150420948
mae_two = 62.5
rmse_two = 235.82302686548658


En general, cabe esperar que el valor del MAE sea mucho menor que el del RMSE. La única diferencia entre los dos conjuntos de errores es el valor extremod de **1000** en **errors_two** en lugar de **10**. Cuando trabajamos con conjuntos de datos mas grandes,  no podemos inspeccionar cada valor para entender si hay uno o algunos **valores atipicos** o si todos los errores son sistematicamente mas altos. Observar la relacion entre el _MAE_ y el _RMSE_ puede ayudarnos a entender si hay errores grandes pero poco frecuentes.